In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
import requests
import re
import time
import pandas as pd

# 使用Safari瀏覽器
driver = webdriver.Safari()
final_list = []

# 到 w3wchools Python 頁面
driver.get("https://www.w3schools.com/python/default.asp")

for i in range(2, 36):
    try:
        # 基於迴圈找到元素
        str_i = str(i)
        xpath = '//*[@id="leftmenuinnerinner"]/a[' + str_i + ']'
        print(xpath)
        element = driver.find_element(By.XPATH, xpath)

        # 點擊該元素
        element.click()

        # 等待頁面加載
        time.sleep(3)
        print(driver.current_url)

        url = driver.current_url

        # 使用正則表達式從URL中提取資訊
        pattern = r"\/([a-z_]+)\.asp"
        match = re.search(pattern, url)
        extracted_text = match.group(1) if match else "No match found"

        print(extracted_text)

        # 設定請求headers
        headers = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
        }

        # 向網頁發送請求
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        sample_t = []
        # 解析網頁內容
        sample = soup.find_all("div", class_="w3-example")
        container_div = soup.find("div", class_="w3-col l10 m12")
        if container_div:
            p_elements = container_div.find_all("p")
            p_elements_t = "\n".join([content.get_text() for content in p_elements])
        for div in sample:
            text = div.text
            index = text.find(">>>")
            if index != -1:
                text = text[index + 3:]

            # 使用正則表達式移除多餘的空格和換行
            text = re.sub(r'\s+', ' ', text)
            sample_t.append(text)
        # 內文p_elements_t
        # 程式sample_t
        print(p_elements_t)
        print("-" * 50)
        sample_t

        # 將提取的資訊添加到列表中
        final_list.append(extracted_text + p_elements_t)
        # 等待時間
        time.sleep(3)
    except NoSuchElementException:
        # 若無法找到元素則輸出並繼續
        print("Element not found for XPath: " + xpath)
        continue

# 關閉瀏覽器
print(final_list)

# 將結果存為CSV檔案
df = pd.DataFrame(final_list)
df.to_csv(f"w3wchools_selenium.csv", index=False)

# 結束瀏覽器進程
driver.quit()